# OpenAI Image Generation - DALL.E 2

_Lukas Schönmann_  
_2022-11-28_  

Objective: generate images to use on the new department landing page; 2 landscape and one square image. We must have the rights to use them.  
Method: OpenAI's DALL.E image generation API

## Setup

Run the cell below once. 

In [1]:
import openai
from configparser import ConfigParser
from IPython.display import Image
import os
import requests

# Read API key from config file
config = ConfigParser()
config.read('config.txt')
openai.api_key = config.get('OPENAI', 'api_key')

# Create directories and files if they don't exist
if not os.path.exists('images'):
    os.makedirs('images')
if not os.path.exists('templates'):	
    os.makedirs('templates')
if not os.path.exists('prompts.txt'):
    with open("prompts.txt", "w") as f:
        f.write("")


# --------FUNCTION DEFINITIONS--------

# Append all used prompts to a file
def appendPromptToFile(prompt):
    with open("prompts.txt", "a") as f:
        f.write(prompt + "\n")

## Image Generation

In [40]:
# Prompt
prompt = "watercolor painting of a dna molecule, vibrant colors"
appendPromptToFile(prompt)

# Image Generation
response = openai.Image.create(
    prompt=prompt,
    n=1, 
    size="1024x1024"
)
image_url = response['data'][0]['url']

# Display image
Image(url=image_url)


## Download

Truncate filenames if too long. 

In [41]:
# Save image
res_img = requests.get(image_url)

dir = "images/"
filename = prompt.replace(" ", "_").replace(",", "_") + ".png"
if len(filename) > 70: 
    filename = filename[:70] + "_xxx.png"
# Check if file already exists
# If it does, append a number to the filename
i = 1
while os.path.exists(dir + filename):
    filename = filename + str(i) + ".png"
    i += 1

# Save image
with open(dir + filename, 'wb') as f:
    f.write(res_img.content)

## Variation

Note: image must be square and PNG with size below 4 Mb. 

In [158]:
use_previous_prompt = False
dir = "templates/"
filename = "dna2" + ".png"

if use_previous_prompt: 
    dir = "images/"
    filename = prompt.replace(" ", "_").replace(".", "_") + ".png"
else: 
    prompt = filename.replace(".png", "").replace("_", " ")

response = openai.Image.create_variation(
    image=open(dir + filename, "rb"),
    n=1,
    size="1024x1024"
)
image_url = response['data'][0]['url']

# Display image
Image(url=image_url)

## Utilities

In [19]:
# Replace all commas in files with underscores
dir = "images/"
for filename in os.listdir(dir):
    if len(filename) < 120: 
        os.rename(dir + filename, dir + filename.replace(",", "_"))
    else: 
        print("Filename too large: " + filename)